In [190]:
import pandas as pd
import os
import fitz
import tabula

In [191]:
input = r'C:\David\Desarrollos\02_Procesos\09_Control_de_Saldos\02_Python\01_Input'
output = r'C:\David\Desarrollos\02_Procesos\09_Control_de_Saldos\02_Python\02_Output'

In [192]:
def obtener_numero_paginas_pdf(archivo):
    pdf_document = fitz.open(archivo)
    return pdf_document.page_count

In [200]:
def tratar_df_brutos_de_pdf(max_proc, contable_null, df):
    print(max_proc, contable_null)

    if max_proc == 5 and not contable_null:
        df_columnas_con_nombres = df[['PROC.', 'SUC-AGE', 'NUM OP', 'HORA', 'ORIGEN', 'TIPO', 'CONTABLE']]
        df_columnas_por_ubicacion = df.iloc[:, [1, -2]]
        return pd.concat([df_columnas_con_nombres, df_columnas_por_ubicacion], axis = 1)
    
    if max_proc == 5 and contable_null:
        df = df.drop(columns = ['CONTABLE'])
        df = df.rename(columns = {df.columns[-1]: 'CONTABLE'})
        df_columnas_con_nombres = df[['PROC.', 'SUC-AGE', 'NUM OP', 'HORA', 'ORIGEN', 'TIPO', 'CONTABLE']]
        df_columnas_por_ubicacion = df.iloc[:, [1, -2]]
        return pd.concat([df_columnas_con_nombres, df_columnas_por_ubicacion], axis = 1)
    
    if max_proc != 5 and not contable_null:
        df['Columna 2'] = df['PROC.'].str[5:]
        df['PROC.'] = df['PROC.'].str[:5]
        df_columnas_con_nombres = df[['PROC.', 'SUC-AGE', 'NUM OP', 'HORA', 'ORIGEN', 'TIPO', 'CONTABLE', 'Columna 2']]
        df_columnas_por_ubicacion = df.iloc[:, [-3]]
        return pd.concat([df_columnas_con_nombres, df_columnas_por_ubicacion], axis = 1)
    
    if max_proc != 5 and contable_null:
        #v Inicio de parte no testeada por caso no presentado (se incluye porque el caso en el que max_prox == 5 y contable_null es True sí se ha dado)
        df = df.drop(columns = ['CONTABLE'])
        df = df.rename(columns = {df.columns[-1]: 'CONTABLE'})     
        #^ Fin de parte no testeada por caso no presentado        
        df['Columna 2'] = df['PROC.'].str[5:]
        df['PROC.'] = df['PROC.'].str[:5]
        df_columnas_con_nombres = df[['PROC.', 'SUC-AGE', 'NUM OP', 'HORA', 'ORIGEN', 'TIPO', 'CONTABLE', 'Columna 2']]
        df_columnas_por_ubicacion = df.iloc[:, [-3]]
        return pd.concat([df_columnas_con_nombres, df_columnas_por_ubicacion], axis = 1)

In [204]:
def generar_dataframe_desde_pagina_pdf(archivo, pagina):
    if pagina == 1:
        df = tabula.read_pdf(archivo, pages=f'{pagina}', stream=True, encoding='ISO-8859-1', area = (350, 0, 842, 595))[0]
        max_proc = df['PROC.'].str.len().max()
        contable_null = (pd.isna(df['CONTABLE'].unique())[0] == True)
        df = tratar_df_brutos_de_pdf(max_proc, contable_null, df)
        df = df.rename(columns = {df.columns[-2]: 'DESCRIPCIÓN', df.columns[-1]: 'CARGO_ABONO', 'PROC.': 'FECHA', 'CONTABLE': 'SALDO CONTABLE'})        
        return df[['FECHA', 'DESCRIPCIÓN', 'SUC-AGE', 'NUM OP', 'HORA', 'ORIGEN', 'TIPO', 'CARGO_ABONO', 'SALDO CONTABLE']]
    
    if pagina > 1:
        """
        df = tabula.read_pdf(archivo, pages=f'{pagina}', stream=True, encoding='ISO-8859-1', area = (195, 0, 842, 595))[0]
        df.columns = ['FECHA', 'DESCRIPCIÓN', 'LUGAR', 'SUC-AGE', 'NUM OP', 'HORA', 'ORIGEN', 'TIPO', 'CARGO_ABONO', 'SALDO CONTABLE']
        df = df[~(df['FECHA'].isna())].drop(columns = ['LUGAR'])

        return df
        """
        df = tabula.read_pdf(archivo, pages=f'{pagina}', stream=True, encoding='ISO-8859-1', area = (195, 0, 842, 595))[0]
        df = df[~(df['PROC.'].isna())] #Lee siempre la primera línea (no encabezados) como nulo
        max_proc = df['PROC.'].str.len().max()
        contable_null = (pd.isna(df['CONTABLE'].unique())[0] == True)
        df = tratar_df_brutos_de_pdf(max_proc, contable_null, df)
        df = df.rename(columns = {df.columns[-2]: 'DESCRIPCIÓN', df.columns[-1]: 'CARGO_ABONO', 'PROC.': 'FECHA', 'CONTABLE': 'SALDO CONTABLE'})  
        return df[['FECHA', 'DESCRIPCIÓN', 'SUC-AGE', 'NUM OP', 'HORA', 'ORIGEN', 'TIPO', 'CARGO_ABONO', 'SALDO CONTABLE']]
        

In [195]:
def generar_dataframe_desde_pdf(archivo):

    ruta_archivo = os.path.join(input, archivo)
    numero_paginas_pdf = obtener_numero_paginas_pdf(ruta_archivo)

    dfs_temp = []

    #Es numero_paginas_pdf - 1 porque la última página de los pdf siempre muestra información no relevante
    for pagina in range(numero_paginas_pdf - 1):
        #Es pagina + 1 porque el pdf empieza desde página 1 y el range empieza desde 0

        if pagina == 0:
            df_temp_1 = generar_dataframe_desde_pagina_pdf(ruta_archivo, pagina + 1)
            dfs_temp.append(df_temp_1)

        else:
            df_temp_2 = generar_dataframe_desde_pagina_pdf(ruta_archivo, pagina + 1)
            dfs_temp.append(df_temp_2)

    df = pd.concat(dfs_temp)

    return df

In [196]:
archivos_xls = [file.name for file in os.scandir(input) if file.name.startswith('ci00_excel')]
archivos_pdf = [file.name for file in os.scandir(input) if file.name.endswith('.pdf')]

In [ ]:
for archivo in archivos_pdf:
    empresa_moneda = archivo[-11:-4]

    match empresa_moneda:
        case 'BPO PEN':
            df_BCP_BPO_PEN = generar_dataframe_desde_pdf(archivo)
        case 'BPO USD':
            df_BCP_BPO_USD = generar_dataframe_desde_pdf(archivo)
        case 'SDP USD':
            df_BCP_SDP_USD = generar_dataframe_desde_pdf(archivo)
        case 'SDP PEN':
            df_BCP_SDP_USD = generar_dataframe_desde_pdf(archivo)
        case _:
            print('Se ha detectado un caso desconocido de EMPRESA_MONEDA en los archivos pdf')

In [ ]:
#Lógica suelta Página 1
archivo = os.path.join(input, archivos_pdf[0])
df = tabula.read_pdf(archivo, pages='1', stream=True, encoding='ISO-8859-1', area = (350, 0, 842, 595))[0]
df.head(5)

In [ ]:
generar_dataframe_desde_pdf(archivo)

In [ ]:
archivo = os.path.join(input, archivos_pdf[1])
df = tabula.read_pdf(archivo, pages='1', stream=True, encoding='ISO-8859-1', area = (350, 0, 842, 595))[0]
df.head(5)

In [ ]:
generar_dataframe_desde_pdf(archivo).head(5)

In [ ]:
archivo = os.path.join(input, archivos_pdf[2])
df = tabula.read_pdf(archivo, pages='1', stream=True, encoding='ISO-8859-1', area = (350, 0, 842, 595))[0]
df.head(5)

In [ ]:
generar_dataframe_desde_pdf(archivo).head(5)

In [ ]:
archivo = os.path.join(input, archivos_pdf[3])
df = tabula.read_pdf(archivo, pages='1', stream=True, encoding='ISO-8859-1', area = (350, 0, 842, 595))[0]
df.head(5)

In [ ]:
generar_dataframe_desde_pdf(archivo).head(5)

In [ ]:
archivo = os.path.join(input, archivos_pdf[4])
df = tabula.read_pdf(archivo, pages='1', stream=True, encoding='ISO-8859-1', area = (350, 0, 842, 595))[0]
df.head(5)

In [ ]:
generar_dataframe_desde_pdf(archivo).head(5)

In [ ]:
archivo = os.path.join(input, archivos_pdf[5])
df = tabula.read_pdf(archivo, pages='1', stream=True, encoding='ISO-8859-1', area = (350, 0, 842, 595))[0]
df.head(5)

In [ ]:
generar_dataframe_desde_pdf(archivo).head(5)

In [ ]:
archivo = os.path.join(input, archivos_pdf[6])
df = tabula.read_pdf(archivo, pages='1', stream=True, encoding='ISO-8859-1', area = (350, 0, 842, 595))[0]
df.head(5)

In [ ]:
generar_dataframe_desde_pdf(archivo).head(5)

In [ ]:
archivo = os.path.join(input, archivos_pdf[7])
df = tabula.read_pdf(archivo, pages='1', stream=True, encoding='ISO-8859-1', area = (350, 0, 842, 595))[0]
df.head(5)

In [ ]:
generar_dataframe_desde_pdf(archivo).head(5)

In [ ]:
archivo = os.path.join(input, archivos_pdf[8])
df = tabula.read_pdf(archivo, pages='1', stream=True, encoding='ISO-8859-1', area = (350, 0, 842, 595))[0]
df.head(5)

In [ ]:
generar_dataframe_desde_pdf(archivo).head(5)

In [ ]:
archivo = os.path.join(input, archivos_pdf[9])
df = tabula.read_pdf(archivo, pages='1', stream=True, encoding='ISO-8859-1', area = (350, 0, 842, 595))[0]
df.head(5)

In [ ]:
generar_dataframe_desde_pdf(archivo).head(5)

In [ ]:
archivo = os.path.join(input, archivos_pdf[10])
df = tabula.read_pdf(archivo, pages='1', stream=True, encoding='ISO-8859-1', area = (350, 0, 842, 595))[0]
df.head(5)

In [ ]:
generar_dataframe_desde_pdf(archivo).head(5)

In [ ]:
archivo = os.path.join(input, archivos_pdf[11])
df = tabula.read_pdf(archivo, pages='1', stream=True, encoding='ISO-8859-1', area = (350, 0, 842, 595))[0]
df.head(5)

In [238]:
archivo = os.path.join(input, archivos_pdf[12])
df = tabula.read_pdf(archivo, pages='2', stream=True, encoding='ISO-8859-1', area = (195, 0, 842, 595))[0]
df.head(5)

IndexError: list index out of range

In [236]:
archivo

'C:\\David\\Desarrollos\\02_Procesos\\09_Control_de_Saldos\\02_Python\\01_Input\\EECC_Set2024 SDP USD.pdf'

In [237]:
generar_dataframe_desde_pdf(archivo)

5 False


,FECHA,DESCRIPCIÓN,SUC-AGE,NUM OP,HORA,ORIGEN,TIPO,CARGO_ABONO,SALDO CONTABLE
0,04-09,TRANSF.BCO.BBVAVEN SUC LIMA,191-000,13589.0,09:17,RCJN,2014,"1,200.00","1,355.30"
1,04-09,IMPUESTO ITFINT,-,NaN,NaN,NaN,909,.05-,"1,355.25"
2,05-09,PTMO.LOC.PRE EMBARQUE INT,191-000,151155.0,15:11,NaN,2501,"799,996.50","801,351.75"
3,05-09,IMPUESTO ITFINT,-,NaN,NaN,NaN,909,39.95-,"801,311.80"
4,10-09,A 191 8744772 1TLC,111-065,13242.0,08:50,OBE962,4401,"522,441.32-","278,870.48"
5,10-09,IMPUESTO ITFINT,-,NaN,NaN,NaN,909,26.10-,"278,844.38"
6,13-09,DE INDUSTRIAS OLEAGINO TLC,111-065,278343.0,16:26,OBED84,2401,600.01,"279,444.39"
7,14-09,DE OLEAGINOSAS AMAZONI TLC,111-008,699335.0,11:19,TNP100,2401,"1,100.00","280,544.39"
8,15-09,IMPUESTO ITFINT,-,NaN,NaN,NaN,909,.05-,"280,544.34"
9,16-09,BCR-CONTINENTALINT,111-005,996074.0,09:22,BOLMCQ,4503,"29,814.00-","250,730.34"
